import pandas as pd
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import os 
print(os.listdir())
import pickle

In [ ]:
import pandas as pd
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import os 
print(os.listdir())
import pickle

In [2]:
data = pd.read_csv('dataset.csv',encoding='utf-8',sep=';')

In [3]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from string import punctuation
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pitonhik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Получение текстовой строки из списка слов
def str_corpus(corpus):
    str_corpus = ''
    for i in corpus:
        str_corpus += ' ' + i
    str_corpus = str_corpus.strip()
    return str_corpus
# Получение списка всех слов в корпусе
def get_corpus(data):
    corpus = []
    for phrase in data:
        for word in phrase.split():
            corpus.append(word)
    return corpus
# Получение облака слов
def get_wordCloud(corpus):
    wordCloud = WordCloud(background_color='white',
                              stopwords=STOPWORDS,
                              width=3000,
                              height=2500,
                              max_words=200,
                              random_state=42
                         ).generate(str_corpus(corpus))
    return wordCloud

In [5]:
def remove_punct(text):
    table = {33: ' ', 34: ' ', 35: ' ', 36: ' ', 37: ' ', 38: ' ', 39: ' ', 40: ' ', 41: ' ', 42: ' ', 43: ' ', 44: ' ', 45: ' ', 46: ' ', 47: ' ', 58: ' ', 59: ' ', 60: ' ', 61: ' ', 62: ' ', 63: ' ', 64: ' ', 91: ' ', 92: ' ', 93: ' ', 94: ' ', 95: ' ', 96: ' ', 123: ' ', 124: ' ', 125: ' ', 126: ' '}
    return text.translate(table)

data['Post_clean'] = data['OPISANIE'].map(lambda x: x.lower())
data['Post_clean'] = data['Post_clean'].map(lambda x: remove_punct(x))
data['Post_clean'] = data['Post_clean'].map(lambda x: x.split(' '))
data['Post_clean'] = data['Post_clean'].map(lambda x: [token for token in x if token not in russian_stopwords\
                                                                  and token != " " \
                                                                  and token.strip() not in punctuation])
data['Post_clean'] = data['Post_clean'].map(lambda x: ' '.join(x))
data['TNVED']=data['TNVED'].map(lambda x: int(str(x)[:2]))

In [6]:
data.isnull().sum()

TNVED         0
OPISANIE      0
DATA          0
Post_clean    0
dtype: int64

In [7]:
data

,TNVED,OPISANIE,DATA,Post_clean
0,90,"ЧАСТИ И ПРИНАДЛЕЖНОСТИ АППАРАТУРЫ ПОЗ.9025, НЕ...",07.09.2021,части принадлежности аппаратуры поз 9025 военн...
1,85,"ТРАНСФОРМАТОРЫ, МОЩНОСТЬЮ БОЛЕЕ 1 КВА, НО НЕ Б...",07.09.2021,трансформаторы мощностью 1 ква 16 ква лом элек...
2,84,"КЛАПАНЫ ЗАПОРНЫЕ ИЗ СТАЛИ, НЕ ВОЕННОГО НАЗНАЧЕ...",07.09.2021,клапаны запорные стали военного назначения
3,85,УСТРОЙСТВА СИГНАЛИЗАЦИОННЫЕ ОХРАННЫЕ И УСТРОЙС...,07.09.2021,устройства сигнализационные охранные устройств...
4,34,"ВЕЩЕСТВА ПОВЕРХНОСТНО-АКТИВНЫЕ ОРГАНИЧЕСКИЕ, А...",02.09.2021,вещества поверхностно активные органические ан...
...,...,...,...,...
4177894,27,"МИНЕРАЛЬНОЕ МОТОРНОЕ МАСЛО, ВСЕСЕЗОННОЕ, ДЛЯ Д...",24.10.2021,минеральное моторное масло всесезонное дизельн...
4177895,34,"СИНТЕТИЧЕСКОЕ МОТОРНОЕ МАСЛО, ВСЕСЕЗОННОЕ, ДЛЯ...",24.10.2021,синтетическое моторное масло всесезонное дизел...
4177896,34,"СИНТЕТИЧЕСКОЕ МОТОРНОЕ МАСЛО, ВСЕСЕЗОННОЕ, ДЛЯ...",24.10.2021,синтетическое моторное масло всесезонное дизел...
4177897,34,"МАСЛО ТРАНСМИССИОННОЕ НА СИНТЕТИЧЕСКОЙ ОСНОВЕ,...",24.10.2021,масло трансмиссионное синтетической основе сод...


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(data['Post_clean'], data['TNVED'], test_size=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [9]:
from sklearn.pipeline import Pipeline
# pipeline позволяет объединить в один блок трансформер и модель, что упрощает написание кода и улучшает его читаемость
from sklearn.feature_extraction.text import TfidfVectorizer
# TfidfVectorizer преобразует тексты в числовые вектора, отражающие важность использования каждого слова из некоторого набора слов (количество слов набора определяет размерность вектора) в каждом тексте
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
# линейный классификатор и классификатор методом ближайших соседей
from sklearn import metrics
# набор метрик для оценки качества модели
from sklearn.model_selection import GridSearchCV
# модуль поиска по сетке параметров

In [10]:
sgd_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42))])
knb_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('knb_clf', KNeighborsClassifier(n_neighbors=10))])
sgd_ppl_clf.fit(X_train, y_train)
pickle.dump(sgd_ppl_clf, open('sgd_ppl_clf.pkl', 'wb'))
print('sgd Завершена')
knb_ppl_clf.fit(X_train, y_train)
pickle.dump(knb_ppl_clf, open('knb_ppl_clf.pkl', 'wb'))
print('knb Завершена')

sgd Завершена
knb Завершена


In [11]:
predicted_sgd = sgd_ppl_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

              precision    recall  f1-score   support

          10       0.82      0.95      0.88       657
          11       0.90      0.87      0.89       863
          12       0.74      0.81      0.78       779
          13       0.34      0.76      0.47        80
          14       0.06      1.00      0.11         3
          15       0.79      0.97      0.87       986
          16       0.91      0.92      0.92      2685
          17       0.70      0.86      0.77      1661
          18       0.89      0.85      0.87      3400
          19       0.89      0.94      0.91      3711
          20       0.92      0.89      0.90      4861
          21       0.83      0.85      0.84      3617
          22       0.99      0.96      0.97      9914
          23       0.88      0.89      0.88      1200
          24       0.96      0.98      0.97       444
          25       0.66      0.93      0.78       988
          26       0.64      0.90      0.75       124
          27       0.96    

In [12]:
predicted_sgd = knb_ppl_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

KeyboardInterrupt: 

In [13]:
sgd_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
    ('sgd_clf', SGDClassifier(penalty='elasticnet', class_weight='balanced', random_state=42))])
sgd_ppl_clf.fit(X_train, y_train)
pickle.dump(knb_ppl_clf, open('sgd_ppl_clf_pipline.pkl', 'wb'))
predicted_sgd = sgd_ppl_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

              precision    recall  f1-score   support

          10       0.91      0.52      0.66      1339
          11       0.97      0.49      0.65      1649
          12       0.83      0.41      0.55      1708
          13       0.91      0.11      0.19      1557
          14       0.91      0.07      0.13       689
          15       0.88      0.44      0.59      2449
          16       0.93      0.71      0.80      3537
          17       0.77      0.69      0.73      2244
          18       0.89      0.76      0.82      3772
          19       0.79      0.86      0.83      3619
          20       0.84      0.77      0.80      5072
          21       0.76      0.75      0.76      3810
          22       0.96      0.91      0.93     10167
          23       0.90      0.55      0.68      1986
          24       0.99      0.30      0.47      1468
          25       0.68      0.42      0.52      2235
          26       0.93      0.14      0.24      1154
          27       0.89    

In [14]:
predicted_sgd_val = sgd_ppl_clf.predict(X_valid)
print(metrics.classification_report(predicted_sgd_val, y_valid))

              precision    recall  f1-score   support

          10       0.92      0.50      0.64       734
          11       0.97      0.55      0.70       882
          12       0.82      0.40      0.54       978
          13       0.92      0.10      0.18       832
          14       0.85      0.06      0.11       383
          15       0.89      0.42      0.57      1408
          16       0.93      0.72      0.81      2003
          17       0.79      0.69      0.74      1259
          18       0.88      0.75      0.81      1990
          19       0.80      0.86      0.83      1948
          20       0.83      0.77      0.80      2875
          21       0.75      0.75      0.75      2056
          22       0.96      0.91      0.93      5685
          23       0.92      0.56      0.70      1151
          24       0.99      0.33      0.49       770
          25       0.68      0.40      0.51      1204
          26       0.90      0.12      0.22       657
          27       0.89    